In [5]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten

In [30]:
model = Sequential()
#convolutional
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 18496)             0         
__________

In [9]:
batch_size = 16

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory('male_female/train', target_size=(150, 150),
                                                    batch_size=batch_size, class_mode='binary')
validation_generator = test_datagen.flow_from_directory('male_female/validation', target_size=(150, 150),
                                                        batch_size=batch_size, class_mode='binary')

Found 1960 images belonging to 2 classes.
Found 840 images belonging to 2 classes.


In [13]:
callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, 
                                       write_grads=False, write_images=False, embeddings_freq=0, 
                                       embeddings_layer_names=None, embeddings_metadata=None)

In [31]:
model.fit_generator(train_generator, steps_per_epoch=1960//batch_size, epochs=30,
                    validation_data=validation_generator, validation_steps=840//batch_size, callbacks=[callback])

Epoch 1/30
122/122 [==============================] - 121s 989ms/step - loss: 0.4942 - acc: 0.7843 - val_loss: 0.3023 - val_acc: 0.8834
Epoch 2/30
122/122 [==============================] - 112s 915ms/step - loss: 0.2820 - acc: 0.8929 - val_loss: 0.3249 - val_acc: 0.8942
Epoch 3/30
122/122 [==============================] - 112s 915ms/step - loss: 0.2405 - acc: 0.9093 - val_loss: 0.2395 - val_acc: 0.9255
Epoch 4/30
122/122 [==============================] - 111s 912ms/step - loss: 0.2136 - acc: 0.9185 - val_loss: 0.2859 - val_acc: 0.8786
Epoch 5/30
122/122 [==============================] - 111s 907ms/step - loss: 0.1747 - acc: 0.9293 - val_loss: 0.2870 - val_acc: 0.8930
Epoch 6/30
122/122 [==============================] - 111s 908ms/step - loss: 0.1792 - acc: 0.9396 - val_loss: 0.3097 - val_acc: 0.9087
Epoch 7/30
122/122 [==============================] - 111s 910ms/step - loss: 0.1482 - acc: 0.9549 - val_loss: 0.2467 - val_acc: 0.9183
Epoch 8/30
122/122 [============================

# Benchmark

1, conv 64 'relu', layer 64 'relu', layer 1 'sigmoid' 
   epocha 10, batch size 16
   loss: 0.1637 - acc: 0.9467 - val_loss: 0.2429 - val_acc: 0.9135

2, conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 10, batch size 16
   loss: 0.1403 - acc: 0.9595 - val_loss: 0.2117 - val_acc: 0.9026
   
3, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 10, batch size 16
   loss: 0.1287 - acc: 0.9513 - val_loss: 0.2827 - val_acc: 0.9038
   
4, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 20, batch size 16
   loss: 0.0752 - acc: 0.9744 - val_loss: 0.2794 - val_acc: 0.9363
   
### 5, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
###     epocha 30, batch size 16
###     loss: 0.0750 - acc: 0.9785 - val_loss: 0.2379 - val_acc: 0.9303
   
6, conv 32 'relu', conv 32 'relu', conv 64 'relu', layer 64 'relu', layer 1 'sigmoid'
   epocha 50, batch size 16
   loss: 0.0984 - acc: 0.9810 - val_loss: 0.8020 - val_acc: 0.9327

In [32]:
model.save_weights('male_female_recognition_model.h5')